In [4]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [3]:
import torch
import matplotlib.pyplot as plt

In [2]:
print(torch.__version__)

2.0.1+cu117


In [4]:
print(torch.cuda.is_available())

True


In [1]:
#Lesson 1

In [11]:
torch.manual_seed(10)
flag = True
if flag:
    w = torch.tensor([1.],requires_grad = True)
    x = torch.tensor([2.],requires_grad = True)
    #y = (w+x)*(w+1)
    a = torch.add(w,x)
    b = torch.add(w,1)
    y = torch.mul(a,b)
    
    y.backward(retain_graph = True)
    y.backward()
    print(w.grad)
    

tensor([10.])


In [13]:
flag = True

w = torch.tensor([1.],requires_grad = True)#w=1
x = torch.tensor([2.],requires_grad = True)#x=2

a = torch.add(w,x)
b = torch.add(w,1)
#y = torch.mul(a,b)

y0 = torch.mul(a,b)
y1 = torch.add(a,b)

loss = torch.cat([y0,y1],dim=0)#y0->5,y1->2
grad_tensors = torch.tensor([1.,1.])#5*1+2*1 = 7
loss.backward(gradient = grad_tensors)
print(w.grad)


tensor([7.])


In [15]:
x = torch.tensor([3.],requires_grad = True)
y = torch.pow(x,2) #y = x^2
grad_1 = torch.autograd.grad(y,x,create_graph = True) #grad_1 = dy/dx = 2x = 2*3 = 6
print(grad_1)
grad_2 = torch.autograd.grad(grad_1[0],x)
print(grad_2)

(tensor([6.], grad_fn=<MulBackward0>),)
(tensor([2.]),)


In [21]:
x = torch.tensor([2.],requires_grad = True)
w = torch.tensor([1.],requires_grad = True)

for i in range(4):
    a = torch.add(w,x)
    b = torch.add(w,1)
    y = torch.mul(a,b)
    
    y.backward()
    print(w.grad)
    
    w.grad.zero_()

tensor([5.])
tensor([5.])
tensor([5.])
tensor([5.])


In [22]:
a = torch.ones((1,))
a = a + torch.ones((1,))#different id
a += torch.ones((1,)) #in-place 
print(id(a),a)

1814655904208 tensor([2.])


In [29]:
#create data
sample_nums = 100
mean_value = 1.7
bias = 1
n_data = torch.ones(sample_nums,2)
x0 = torch.normal(mean_value*n_data,1) + bias
y0 = torch.zeros(sample_nums)
x1 = torch.normal(-mean_value*n_data,1) + bias
y1 = torch.ones(sample_nums)
train_x = torch.cat((x0,x1),0)
train_y = torch.cat((y0,y1),0)


In [30]:
#choose model
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.features = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        x = self.features(x)
        x = self.sigmoid(x)
        return x
    
lr_net = LR()


##choose loss function

loss_fn = nn.BCELoss()

###select optimizer
lr = 0.01
optimizer = torch.optim.SGD(lr_net.parameters(),lr = lr, momentum = 0.9)


### Train the model
for iteration in range(1000):
    y_pred = lr_net(train_x)
    loss = loss_fn(y_pred.squeeze(),train_y)
    loss.backward()
    optimizer.step()
    
    if iteration % 20 == 0:
        mask = y_pred.ge(0.5).float().squeeze()#以0.5为阈值进行分类
        correct = (mask == train_y).sum()#计算正确预测的样本个数
        acc = correct.item()/train_y.size(0)#计算分类准确率
        
        plt.scatter(x0.data.numpy()[:,0],x0.data.numpy()[:,1],c='r',label = 'class 0')
        plt.scatter(x1.data.numpy()[:,0],x1.data.numpy()[:,1],c='b',label = 'class 1')
        
        w0,w1 = lr.net.features.weight[0]
        

SyntaxError: incomplete input (3680323351.py, line 26)

In [12]:
torch.manual_seed(10)

In [18]:
lr = 0.1
x = torch.rand(20,1)*10
#print(x)
y = 2*x + (5+torch.randn(20,1))

w = torch.randn((1),requires_grad = True)
b = torch.zeros((1),requires_grad = True)

for iteration in range(100):
    wx = torch.mul(w,x)
    y_pred = torch.add(wx,b)
    loss = (0.5*(y-y_pred)**2).mean()
    
    loss.backward()

    b.data.sub_(lr*b.grad)
   
    w.data.sub_(lr*w.grad)
    
    
    if loss.data.numpy()<1:
        print('w:{}'.format(w))
        print('b:{}'.format(b))
        print('loss=%.4f'%loss.data.numpy())
        #plt.scatter(x.data.numpy(),y.data.numpy())
        #plt.plot(x.data.numpy(),y_pred.data.numpy(),'r-',lw=5)
        
        break




w:tensor([10.0638], requires_grad=True)
b:tensor([4.3294], requires_grad=True)
loss=0.6799


In [17]:
    if iteration %20 == 0:
        plt.scatter(x.data.numpy(),y.data.numpy())
        plt.plot(x.data.numpy(),y_pred.data.numpy(),'r-',lw=5)